In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [4]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)
driver.close()

In [12]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [13]:
games_df

,Winner,Loser
0,ATL,GSW
1,SAC,CHI
2,MIL,DAL
3,LAL,NYK
4,BRK,PHI
5,CLE,SAS
